In [2]:
import csv
import numpy as np
import pandas as pd
import itertools
import folium
import time
import openrouteservice as ors
import sys
import urllib.parse
import pulp
import datetime

filename = 'test1.csv'
with open(filename, encoding='utf8', newline='') as f:
    csvreader = csv.reader(f)
    place_list = [row for row in csvreader]
place_list

[['地名', 'latitude', 'longitude'],
 ['長津田駅', '35.5319589', '139.49173'],
 ['河口湖ビジネス&リゾート+サワホテル', '35.4946247', '138.7589732'],
 ['Bakery&Table+箱根（ベーカリー＆テーブル）', '35.2001708', '139.0321096'],
 ['うなぎ+桜家', '35.1191933', '138.9138269'],
 ['箱根神社', '35.2040652', '139.0266838'],
 ['大涌谷', '35.2414681', '139.0227329'],
 ['早雲山駅', '35.2459732', '139.0357894'],
 ['箱根ガラスの森美術館', '35.2609093', '139.0183711'],
 ['三嶋大社', '35.1195374', '138.9178982'],
 ['三島スカイウォーク', '35.1547315', '138.9820768']]

datalist = []

ors_key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
ors_boot = ors.Client(ors_key)

for i in range(1,len(place_list)-1):
    pos_1 = float(place_list[i][1]),float(place_list[i][2])
    for j in range(2,len(place_list)):
        pos_2 = float(place_list[j][1]),float(place_list[j][2])
        p1r = tuple(reversed(pos_1))
        p2r = tuple(reversed(pos_2))
        
        try:
            route_rslt = ors_boot.directions((p1r,p2r),profile = "driving-car")
            datalist.append([i,j,float(route_rslt["routes"][0]["summary"]["duration"])])
            print(i,j,float(route_rslt["routes"][0]["summary"]["duration"]))
        except:
            pass
tfr_time = pd.DataFrame(datalist, columns=["出発地点","行先","移動時間[s]"])


tfr_time.to_csv("time1.csv",index=0)

In [4]:
tfr_time = pd.read_csv("time1.csv")

customer_count = len(place_list)-1
loc_loc_time = tfr_time.to_numpy().tolist()
num_places_time = len(tfr_time)

cost = [[86401 for i in range(customer_count)] for j in range(customer_count)]
for i in range(num_places_time):
    cost[int(loc_loc_time[i][0])-1][int(loc_loc_time[i][1])-1] = float(loc_loc_time[i][2])
    cost[int(loc_loc_time[i][1])-1][int(loc_loc_time[i][0])-1] = float(loc_loc_time[i][2])
print(cost)
cost = np.array(cost)
cost

[[86401, 4969.9, 4580.9, 4332.1, 4617.9, 86401, 4270.1, 4021.4, 4366.5, 4772.3], [4969.9, 86401, 3461.0, 2888.2, 3579.9, 86401, 2895.7, 2647.1, 2922.6, 3327.4], [4580.9, 3461.0, 86401, 1443.9, 311.9, 86401, 624.7, 907.1, 1430.5, 1105.5], [4332.1, 2888.2, 1443.9, 86401, 1651.7, 86401, 1981.6, 2035.2, 34.4, 656.9], [4617.9, 3579.9, 311.9, 1651.7, 86401, 86401, 661.7, 944.1, 1529.5, 1313.5], [86401, 86401, 86401, 86401, 86401, 86401, 86401, 86401, 86401, 86401], [4270.1, 2895.7, 624.7, 1981.6, 661.7, 86401, 86401, 341.9, 1968.3, 1643.3], [4021.4, 2647.1, 907.1, 2035.2, 944.1, 86401, 341.9, 86401, 2072.4, 1925.7], [4366.5, 2922.6, 1430.5, 34.4, 1529.5, 86401, 1968.3, 2072.4, 86401, 643.6], [4772.3, 3327.4, 1105.5, 656.9, 1313.5, 86401, 1643.3, 1925.7, 643.6, 86401]]


array([[8.6401e+04, 4.9699e+03, 4.5809e+03, 4.3321e+03, 4.6179e+03,
        8.6401e+04, 4.2701e+03, 4.0214e+03, 4.3665e+03, 4.7723e+03],
       [4.9699e+03, 8.6401e+04, 3.4610e+03, 2.8882e+03, 3.5799e+03,
        8.6401e+04, 2.8957e+03, 2.6471e+03, 2.9226e+03, 3.3274e+03],
       [4.5809e+03, 3.4610e+03, 8.6401e+04, 1.4439e+03, 3.1190e+02,
        8.6401e+04, 6.2470e+02, 9.0710e+02, 1.4305e+03, 1.1055e+03],
       [4.3321e+03, 2.8882e+03, 1.4439e+03, 8.6401e+04, 1.6517e+03,
        8.6401e+04, 1.9816e+03, 2.0352e+03, 3.4400e+01, 6.5690e+02],
       [4.6179e+03, 3.5799e+03, 3.1190e+02, 1.6517e+03, 8.6401e+04,
        8.6401e+04, 6.6170e+02, 9.4410e+02, 1.5295e+03, 1.3135e+03],
       [8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04,
        8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04, 8.6401e+04],
       [4.2701e+03, 2.8957e+03, 6.2470e+02, 1.9816e+03, 6.6170e+02,
        8.6401e+04, 8.6401e+04, 3.4190e+02, 1.9683e+03, 1.6433e+03],
       [4.0214e+03, 2.6471e+03, 9.0710e+0

In [10]:
start_place_time = (datetime.time(9,00),datetime.time(17,00))
lunch_time = datetime.time(12,00)
lim_time = (datetime.datetime.combine(datetime.date.today(), lunch_time) -\
    datetime.datetime.combine(datetime.date.today(), start_place_time[0])).seconds
count_place = customer_count
lunch_count = 2

count_remove_place = list(range(0,count_place))
del count_remove_place[2:lunch_count+2]
del count_remove_place[0]
    

print(count_remove_place)

# 問題の宣言
problem = pulp.LpProblem("CVRP", pulp.LpMaximize)

# 決定変数
x = [[pulp.LpVariable("x%s_%s"%(i,j), cat="Binary") if i != j else None for j in range(count_place)] for i in range(count_place)]

# 目的関数
# なるたけ訪れる地点を多くする
problem += pulp.lpSum(x[i][j] if i != j else 0
                        for j in range(count_place) for i in range (count_place))


# 制約
# (2)式，各地点に訪れるのは１回を超えてはならない
for j in range(1, count_place):
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) <= 1 

#(3)式, 出発地から始まり，指定した昼ご飯の場所に到着する

# 出発地から必ず出る
problem += pulp.lpSum(x[0][j] for j in range(1,count_place)) == 1
# 指定した昼ご飯の場所に到着する
problem += pulp.lpSum(x[i][k] for i in range(count_place) for k in range(2,lunch_count + 2)) == 1         #############ここを変える
problem += pulp.lpSum(x[k][j] for j in range(count_place) for k in range(2,lunch_count + 2)) == 0         #############ここを変える

problem += pulp.lpSum(x[i][0] for i in range(1,count_place)) == 0

#(4)式, 出発到着地点以外の箇所に対し出入りの数が同じ

for j in count_remove_place:         #############ここを変える
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i] for i in range(count_place)) == 0

#(5)式, 時間制約を越えない

problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j]if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= lim_time +7200
problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j]if i != j else 0 for i in range(count_place) for j in range (1,count_place)) >= lim_time +3600

#(6)式, 部分巡回路除去制約
subtours = []
for i in range(2,count_place):
        subtours += itertools.combinations(range(1,count_place), i)

for s in subtours:
    problem += pulp.lpSum(x[i][j] if i !=j else 0 for i, j in itertools.permutations(s,2)) <= len(s) - 1


if problem.solve() == 1:
    print('目的関数値:', pulp.value(problem.objective))

[1, 4, 5, 6, 7, 8, 9]
目的関数値: 3.0


In [11]:
basyo_num_list = []
basyo_list = [0]
jikan = 0


for i in range(customer_count):
    for j in range(customer_count):
        if i != j and pulp.value(x[i][j]) == 1:
            print(i,j)
            jikan += 3600 + cost[i][j]
            basyo_num_list.append([i,j])


print(basyo_num_list)
print("時間",lim_time/3600,"~",(lim_time + 3600)/3600,"\n最適化",(jikan-3600)/3600)

for i in range(len(basyo_num_list)):
    for j in range(len(basyo_num_list)):
        if basyo_num_list[j][0] == basyo_list[-1]:
            basyo_list.append(basyo_num_list[j][1])
basyo_list

0 9
8 3
9 8
[[0, 9], [8, 3], [9, 8]]
時間 3.0 ~ 4.0 
最適化 3.513972222222222


[0, 9, 8, 3]

In [9]:
basyo_list1 = basyo_list.copy()
# いったとこlist
del basyo_list1[0]
del basyo_list1[-1]
print(basyo_list1)

# 新たな目的地
count_place2 = list(range(0,count_place))
del count_place2[2:lunch_count+2]
for i in basyo_list1:
    count_place2.remove(i)
count_place2.append(2)
count_place2.sort()

count_place2

[9, 8]


[0, 1, 2, 4, 5, 6, 7]

In [ ]:
start_place_time = (datetime.time(8,00),datetime.time(17,00))
lim_time = (datetime.datetime.combine(datetime.date.today(), start_place_time[1])).seconds - \
    jikan-3600

In [ ]:
start_place_time = (datetime.time(8,00),datetime.time(17,00))
lim_time = datetime.datetime.combine(datetime.date.today(), start_place_time[1]).seconds - \
    jikan-3600
count_place = customer_count
lunch_count = 2

count_remove_place = list(range(0,count_place))
del count_remove_place[2:lunch_count+2]
del count_remove_place[0]
    

print(count_remove_place)

# 問題の宣言
problem = pulp.LpProblem("CVRP", pulp.LpMaximize)

# 決定変数
x = [[pulp.LpVariable("x%s_%s"%(i,j), cat="Binary") if i != j else None for j in range(count_place)] for i in range(count_place)]

# 目的関数
# なるたけ訪れる地点を多くする
problem += pulp.lpSum(x[i][j] if i != j else 0
                        for j in range(count_place) for i in range (count_place))


# 制約
# (2)式，各地点に訪れるのは１回を超えてはならない
for j in range(1, count_place):
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) <= 1 

#(3)式, 出発地から始まり，指定した昼ご飯の場所に到着する

# 出発地から必ず出る
problem += pulp.lpSum(x[0][j] for j in range(1,count_place)) == 1
# 指定した昼ご飯の場所に到着する
problem += pulp.lpSum(x[i][k] for i in range(count_place) for k in range(2,lunch_count + 2)) == 1         #############ここを変える
problem += pulp.lpSum(x[k][j] for j in range(count_place) for k in range(2,lunch_count + 2)) == 0         #############ここを変える

problem += pulp.lpSum(x[i][0] for i in range(1,count_place)) == 0

#(4)式, 出発到着地点以外の箇所に対し入出の数が同じ

for j in count_remove_place:         #############ここを変える
    problem += pulp.lpSum(x[i][j] if i != j else 0 for i in range(count_place)) -  pulp.lpSum(x[j][i] for i in range(count_place)) == 0

#(5)式, 時間制約を越えない

problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j]if i != j else 0 for i in range(count_place) for j in range (1,count_place)) <= lim_time +7200
problem += pulp.lpSum((3600 + cost[i][j]) * x[i][j]if i != j else 0 for i in range(count_place) for j in range (1,count_place)) >= lim_time +3600

#(6)式, 部分巡回路除去制約
subtours = []
for i in range(2,count_place):
        subtours += itertools.combinations(range(1,count_place), i)

for s in subtours:
    problem += pulp.lpSum(x[i][j] if i !=j else 0 for i, j in itertools.permutations(s,2)) <= len(s) - 1


if problem.solve() == 1:
    print('目的関数値:', pulp.value(problem.objective))
    print(problem)